In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F 

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
import pandas as pd

In [2]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1) 

In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [4]:
dtype = torch.float32 
device = torch.device('cuda')

In [5]:
# transform  the 1-D data to  2-D  image data form:
df=pd.read_csv('train.csv')

In [6]:
X=df.to_numpy()
Y=X[:,0]
X=X[:,1:785]

In [7]:
X_train=X[0:39000]
Y_train=Y[:39000]
X_val=X[39000:42000]
Y_val=Y[39000:42000]

In [8]:
X_train=X_train.reshape((39000,1,28,28))
X_val=X_val.reshape((3000,1,28,28))

In [9]:
X_train=torch.tensor(X_train)
X_val=torch.tensor(X_val)
y_train=torch.tensor(Y_train)
y_val=torch.tensor(Y_val)

In [10]:
X_train=X_train.to(device=device, dtype=dtype)
y_train=y_train.to(device=device, dtype=torch.long)
X_val=X_val.to(device=device, dtype=dtype)
y_val=y_val.to(device=device, dtype=torch.long)

In [11]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print("Epoch%d:"%(e+1))
        for t in range(390):
            model.train()  # put model to training mode
            batch_mask=np.random.choice(39000,100)
            x=X_train[batch_mask]
            y=y_train[batch_mask]
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                #check_accuracy_part34(loader_val, model)
                #print()

In [12]:
learning_rate = 1e-3
model = nn.Sequential(
    nn.Conv2d(1,64,(3,3),padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(64,64,(3,3),padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    
    nn.BatchNorm2d(64),
    nn.Conv2d(64,128,(3,3),padding=1),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(128),
    nn.Conv2d(128,128,(3,3),padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),  # 128*7*7  
    
    
    nn.BatchNorm2d(128),
    
    Flatten(),
    nn.Linear(128*7*7,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128,10),
)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5
                 )


In [13]:
train_part34(model, optimizer, epochs=10)

Epoch1:
Iteration 0, loss = 2.5833
Iteration 100, loss = 0.0249
Iteration 200, loss = 0.0495
Iteration 300, loss = 0.0156
Epoch2:
Iteration 0, loss = 0.1215
Iteration 100, loss = 0.0443
Iteration 200, loss = 0.0192
Iteration 300, loss = 0.1040
Epoch3:
Iteration 0, loss = 0.0417
Iteration 100, loss = 0.0036
Iteration 200, loss = 0.0118
Iteration 300, loss = 0.0035
Epoch4:
Iteration 0, loss = 0.0084
Iteration 100, loss = 0.0068
Iteration 200, loss = 0.0057
Iteration 300, loss = 0.0049
Epoch5:
Iteration 0, loss = 0.0019
Iteration 100, loss = 0.0074
Iteration 200, loss = 0.0142
Iteration 300, loss = 0.0016
Epoch6:
Iteration 0, loss = 0.0008
Iteration 100, loss = 0.0195
Iteration 200, loss = 0.0020
Iteration 300, loss = 0.0058
Epoch7:
Iteration 0, loss = 0.0009
Iteration 100, loss = 0.0011
Iteration 200, loss = 0.0009
Iteration 300, loss = 0.0077
Epoch8:
Iteration 0, loss = 0.0085
Iteration 100, loss = 0.1473
Iteration 200, loss = 0.0007
Iteration 300, loss = 0.0057
Epoch9:
Iteration 0, los

In [14]:
def check_accuracy_part34(model):
    
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
            scores = model(X_val)
            _, preds = scores.max(1)
            num_correct += (preds == y_val).sum()
            num_samples += preds.size(0)
            acc = float(num_correct) / num_samples
            print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [15]:
check_accuracy_part34(model)

Got 2982 / 3000 correct (99.40)
